# short_test.ipynb
Functions to examine rolling correlations between device sensor outputs.
Author: – Jon Clucas, 2017 jon.clucas@childmind.org
© 2017, Child Mind Institute, Apache v2.0 License

setup:

In [ ]:
%matplotlib inline
from annotate_range import annotation_line
from astropy.stats import median_absolute_deviation as mad
from chart_data import write_csv
from config import short_dir, cache_hashes, test_urls
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from normalize_acc_data import actigraph_acc, geneactiv_acc
from plot_normalized_vector_lengths import baseshift_and_renormalize
from utilities.fetch_data import fetch_check_data, fetch_data, fetch_hash
import json, numpy as np, os, pandas as pd, matplotlib.pyplot as plt
with open(os.path.join('./line_charts/device_colors.json')) as fp:
    color_key = json.load(fp)
pd.set_option('mode.use_inf_as_null', True)
hashes = cache_hashes()
if not os.path.exists('./sample_data'):
    os.makedirs('./sample_data')

In [ ]:
acc_hashes = {}

In [ ]:
for ppg in ['e4_ppg', 'Wavelet_ppg']:
    try:
        fetch_check_data(ppg, test_urls()[ppg], hashes, cache_directory='./sample_data', append='.csv', verbose=True)
    except OSError:
        hashes[ppg] = fetch_hash(fetch_data(test_urls()[ppg], os.path.join('./sample_data', ppg), '.csv'))

In [ ]:
print(hashes)

define functions:

In [ ]:
def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')

def df_devices(devices, sensor, start, stop):
    """
    Function to calculate rolling correlations between two sensor data streams.
    
    Parameters
    ----------
    devices : list of (subdirectory, device) tuples (len 2)
        each string is the name of one of the two devices to compare
        
    sensor : string
        the sensor to compare
        
    start : datetime
        beginning of time to compare
        
    stop : datetime
        end of time to compare
        
    Returns
    -------
    df : pandas dataframe
        merged dataframe with a column per device
    """
    suffix = '.csv'
    s = []
    for i, device in enumerate(devices):
        acc_sub = '_'.join([device[0], 'acc', 'quicktest'])
        if not acc_sub in acc_hashes:
            try:
                fetch_check_data(acc_sub, test_urls()[acc_sub], acc_hashes, cache_directory='./sample_data',
                                 append='.csv', verbose=True)
            except OSError:
                acc_hashes[acc_sub] = fetch_data(test_urls()[acc_sub], os.path.join('./sample_data', acc_sub), '.csv')
        s.append(pd.read_csv(os.path.join('./sample_data', ''.join([acc_sub, suffix])),
                 usecols=['Timestamp', 'normalized_vector_length'],
                 parse_dates=['Timestamp'], infer_datetime_format=True))
        s[i] = s[i].loc[(s[i]['Timestamp'] >= start) & (s[i]['Timestamp'] <= stop)].copy()
        s[i] = baseshift_and_renormalize(s[i])
        if device[1] == 'ActiGraph':
            s[i][['Timestamp']] = s[i].Timestamp.apply(lambda x: x - timedelta(microseconds
                        =1000))
        s[i].set_index('Timestamp', inplace=True)
    df = s[0].merge(s[1], left_index=True, right_index=True, suffixes=(''.join([
         '_', devices[0][1]]), ''.join(['_', devices[1][1]])))
    for i in range(2, len(s), 1):
        df = df.merge(s[i], left_index=True, right_index=True, suffixes=('', ''.join(['_', devices[i][1]])))
    return(df)
    
def linechart(df, plot_label, line=True, full=False):
    """
    Function to build a linechart and export a PNG and an SVG of the image.
    
    Parameters
    ----------
    df : pandas dataframe
        dataframe to plot
        
    plot_label : string
        plot title
        
    line : boolean
        True for lineplot, False for scatterplot
        
    full : boolean
        True for ylim=[0, 1], False for ylim=[0, 3×max(mad)
        
    Returns
    -------
    plotted : boolean
        True if data plotted, False otherwise
    
    Outputs
    -------
    inline plot
    """
    try:
        start = min(df.index.values)
    except:
        print("End of data.")
        return False
    stop = max(df.index.values)
    print("Plotting...")
    print(plot_label)
    fig = plt.figure(figsize=(10, 8), dpi=75)
    plt.rcParams['agg.path.chunksize'] = 10000
    ax = fig.add_subplot(111)
    ax.set_ylabel('unit cube normalized vector length')
    annotations_a = {}
    annotations_b = {}
    annotation_y = 0.04
    mad_values = []
    for device in list(df.columns):
        if device.startswith('normalized'):
            d2 = device[25:]
        else:
            d2 = device
        plot_line = df[[device]].dropna()
        mp = mad(plot_line)
        if mp > 0:
            print(mp)
            mad_values.append(mp)
        else:
            mp = plot_line.std()[0]
            if mp > 0:
                print(mp)
                mad_values.append(mp)
            else:
                print(max(plot_line[[device]]))
                mad_values.append(max(plot_line[[device]]))
        if "GENEActiv" in device:
            label = "GENEActiv"
        elif device == "Actigraph":
            label = "ActiGraph"
        else:
            label = d2
        """
        if device == "Wavelet":
            ax.plot_date(x=plot_line.index, y=plot_line, color=color_key[
                         device], alpha=0.5, label=label, marker="o",
                         linestyle="None")
        else:
        """
        if line:
            ax.plot_date(x=plot_line.index, y=plot_line, alpha=0.5, label=label, marker="", linestyle=
                             "solid")
        else:
            ax.plot_date(x=plot_line.index, y=plot_line, alpha=0.5, label=label, marker="o", linestyle=
                             "None")
        ax.legend(loc='best', fancybox=True, framealpha=0.5)
    try:
        ylim = max(mad_values)
    except:
        ylim = 0
    if full or ylim == 0:
        ax.set_ylim([0, 1])
    else:
        try:
            ax.set_ylim([0, 3 * ylim])
        except:
            ax.set_ylim([0, 1])
    ax.xaxis.set_major_formatter(DateFormatter('%H:%M:%S'))
    plt.suptitle(plot_label)
    plt.xticks(rotation=65)
    plt.show()
    return True
    
def rolling_window(a, window):
    # http://wichita.ogs.ou.edu/documents/python/xcor.py
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
    
def xcorr(x,y):
  """c=xcor(x,y)
  Fast implementation to compute the normalized cross correlation where x and y are 1D numpy arrays
  x is the timeseries
  y is the template time series
  returns a numpy 1D array of correlation coefficients, c"
  
  The standard deviation algorithm in numpy is the biggest slow down in this method.  
  The issue has been identified hopefully they make improvements.

  http://wichita.ogs.ou.edu/documents/python/xcor.py
  """
  N=len(x)
  M=len(y)
  meany=np.mean(y)
  stdy=np.std(np.asarray(y))
  tmp=rolling_window(x,M)
  c=np.sum((y-meany)*(tmp-np.reshape(np.mean(tmp,-1),(N-M+1,1))),-1)/(M*np.std(tmp,-1)*stdy)

  return c

normalize data:

In [ ]:
a1 = actigraph_acc(os.path.join(short_dir, 'A'), os.path.join(short_dir, 'A'))

In [ ]:
g1 = geneactiv_acc(os.path.join(short_dir, 'G1'), os.path.join(short_dir, 'G1'))

In [ ]:
g2 = geneactiv_acc(os.path.join(short_dir, 'G2'), os.path.join(short_dir, 'G2'))

load normalized data:

In [ ]:
df = df_devices([('A', 'ActiGraph'), ('G1', 'GENEActiv'), ('G2', 'GENEActiv')], 'accelerometer',
     datetime(2017, 4, 28, 15, 30), datetime(2017, 4, 28, 15, 48))
df.rename(columns={'normalized_vector_length': 'normalized_vector_length_GENEActiv(2)'}, inplace=True)

In [ ]:
linechart(df, 'ActiGraph vs 2×GENEActiv', line=True, full=True)

In [ ]:
linechart(df, 'ActiGraph vs 2×GENEActiv', line=False, full=True)

In [ ]:
Avalues = df['normalized_vector_length_ActiGraph'].values
G1values = df['normalized_vector_length_GENEActiv'].values
G2values = df['normalized_vector_length_GENEActiv(2)'].values

In [ ]:
shiftG1G2 = len(G1values) - np.argmax(np.correlate(G1values, G2values, mode='full'))
shiftG1A = len(G1values) - np.argmax(np.correlate(G1values, Avalues, mode='full'))
shiftG2A = len(G2values) - np.argmax(np.correlate(G2values, Avalues, mode='full'))
shiftGA = np.int(np.mean([shiftG1A, shiftG2A]))
[shiftG1G2, shiftG1A, shiftG2A, shiftGA]

In [ ]:
shift_GA = np.abs(shiftGA)

In [ ]:
Avalues_shifted = Avalues[:G1values.shape[0]-shift_GA]
G1values_shifted = G1values[shift_GA:G1values.shape[0]]
G2values_shifted = G2values[shift_GA:G2values.shape[0]]
[np.shape(G1values_shifted), np.shape(G2values_shifted), np.shape(Avalues_shifted)]

In [ ]:
[xcorr(G1values_shifted, G2values_shifted), xcorr(Avalues_shifted, G1values_shifted), xcorr(Avalues_shifted, G2values_shifted)]

In [ ]:
shifted_t = [datetime(2017, 4, 28, 15, 30)]
while len(shifted_t) < np.shape(Avalues_shifted)[0]:
    shifted_t.append(shifted_t[-1] + timedelta(seconds=0.0166))
shifted_df = pd.DataFrame({'normalized_vector_length_ActiGraph': Avalues_shifted,
            'normalized_vector_length_GENEActiv': G1values_shifted,
            'normalized_vector_length_GENEActiv(2)': G2values_shifted, 'Timestamp':shifted_t})
shifted_df.set_index('Timestamp', inplace=True)

In [ ]:
start = shifted_t[0]
stop = shifted_t[-1]
while start < stop:
    new_start = start + timedelta(seconds=180)
    plot_df = shifted_df.loc[(shifted_df.index >= start) & (shifted_df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    linechart(plot_df, label, False)
    print(xcorr(plot_df['normalized_vector_length_GENEActiv'].values,
                plot_df['normalized_vector_length_GENEActiv(2)'].values))
    print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
                plot_df['normalized_vector_length_GENEActiv'].values))
    print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
                plot_df['normalized_vector_length_GENEActiv(2)'].values))
    start = new_start

In [ ]:
linechart(shifted_df, 'ActiGraph vs 2×GENEActiv, shifted', line=False, full=True)

In [ ]:
linechart(shifted_df, 'ActiGraph vs 2×GENEActiv, shifted', line=True, full=True)

cut middle portion out when devices were being transferred:

In [ ]:
start1 = datetime(2017,4,28,15,30)
stop1 = datetime(2017,4,28,15,37)
start2 = datetime(2017,4,28,15,40)
stop2 = datetime(2017,4,28,15,48)
cropped_df = shifted_df.loc[(shifted_df.index >= start1) & (shifted_df.index <= stop1) |
                            (shifted_df.index >= start2) & (shifted_df.index <= stop2)].copy()

In [ ]:
linechart(cropped_df, 'ActiGraph vs 2×GENEActiv, shifted, cropped section', line=True, full=True)

In [ ]:
Avalues_cropped = cropped_df['normalized_vector_length_ActiGraph'].values
G1values_cropped = cropped_df['normalized_vector_length_GENEActiv'].values
G2values_cropped = cropped_df['normalized_vector_length_GENEActiv(2)'].values

compute normalized cross-correlations:

In [ ]:
[xcorr(G1values_cropped, G2values_cropped), xcorr(Avalues_cropped, G1values_cropped),
 xcorr(Avalues_cropped, G2values_cropped)]

plot x-second windows:

In [ ]:
start = datetime(2017,4,28,15,30) #shifted_t[0]
stop = datetime(2017,4,28,15,48) #shifted_t[-1]
plot_data = True
while start < stop and plot_data:
    new_start = start + timedelta(seconds=10)
    plot_df = cropped_df.loc[(cropped_df.index >= start) & (cropped_df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    plot_data = linechart(plot_df, label, line=True, full=False)
    #print(xcorr(plot_df['normalized_vector_length_GENEActiv'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv'].values))
    #print(xcorr(plot_df['normalized_vector_length_ActiGraph'].values,
    #            plot_df['normalized_vector_length_GENEActiv(2)'].values))
    start = new_start

np.shape(G1values)

In [ ]:
start = datetime(2017, 4, 28, 15, 29)
stop = datetime(2017, 4, 28, 16, 29)
while start < stop:
    new_start = start + timedelta(seconds=30)
    plot_df = df.loc[(df.index >= start) & (df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    linechart(plot_df, label, False)
    start = new_start

In [ ]:
start = datetime(2017, 4, 28, 15, 29)
stop = datetime(2017, 4, 28, 16, 29)
while start < stop:
    new_start = start + timedelta(seconds=30)
    plot_df = df.loc[(df.index >= start) & (df.index <= new_start)].copy()
    label = '–'.join([start.strftime('%H:%M:%S'), new_start.strftime('%H:%M:%S')])
    linechart(plot_df, label, False, True)
    start = new_start